# Schudeler Query from Bronce to Silver for Champions

Library

In [ ]:
import pandas as pd
from pandas_gbq import read_gbq, to_gbq
import json

Project Info

In [ ]:
project_id = "big-data-lol"
dataset_id_from = "Bronze_LOL_Dataset"
dataset_id_to = "Silver_LOL_Dataset"
table_id_from = "bronze_batch_img_champ"
table_id_to = "silver_batch_champion"

Lol current patch

In [ ]:
current_patch = "14.12"

Query

In [ ]:
query = f'''
SELECT DATA
FROM {project_id}.{dataset_id_from}.{table_id_from}
WHERE TIMESTAMP_TRUNC(_PARTITIONTIME, DAY) = TIMESTAMP("2024-06-16")
AND Patch = "{current_patch}"
'''

Query in BigQuery Dataset

In [ ]:
df = read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [ ]:
def json_to_dataframe(json_str):
    # Cargar el JSON en un diccionario
    json_data = json.loads(json_str)

    # Crear un DataFrame a partir del diccionario
    df = pd.DataFrame.from_dict([json_data])

    return df



In [ ]:
df_processed = pd.concat([json_to_dataframe(row) for row in df['DATA']], ignore_index=True)


Schema create

In [ ]:
from google.cloud import bigquery
schema = [

  bigquery.SchemaField('id', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('key', 'INTEGER', 'REQUIRED'),
  bigquery.SchemaField('name', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('title', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('image', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('skins', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('lore', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('blurb', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('allytips', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('enemytips', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('tags', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('partype', 'STRING', 'NULLABLE'),
  bigquery.SchemaField('info', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('stats', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('spells', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('passive', 'STRING', 'REQUIRED'),
  bigquery.SchemaField('recommended', 'STRING', 'REQUIRED')
]



In [ ]:
client = bigquery.Client(project=project_id)

validated_path = '/tmp/validated_' + "champs_to_silver.csv"
# se transforma el df a csv
df_processed.to_csv(validated_path, index=False)

print(validated_path)
# Cargar los datos validados en BigQuery
dataset_ref = client.dataset(dataset_id_to)
table_ref = dataset_ref.table(table_id_to)

#configuracion del job
job_config = bigquery.LoadJobConfig()

job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.schema = schema
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE


with open(validated_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()


/tmp/validated_champs_to_silver.csv


LoadJob<project=big-data-lol, location=US, id=ce39609d-2138-41d8-bf58-8ac5846aae69>

Job Configuration